# Experiment 3: Variance
This notebook tests the impact of data augmentation and fine tuning on the final results. We notice that data augmentation and fine tuning inject noise into the process. 

Tests:
- Zero-shot Transfer Learning
- Fine tuned Transfer Learning without data augmentation
- Fine Tuned Transfer Learning with data augmentation

Datasets:
- OpenSARShip
- FUSAR-Ship

In [1]:
#Stuff for google colab

In [2]:
import pandas as pd

import utils
from experiments import experiment_3, EXPERIMENT_3_SAVE_PATH, EXPERIMENT_3_NUM_EXPERIMENTS

## Parameters and Function Definitions

In [3]:
#Non-Default Parameters

#(data augmentation, embedding)
test_list = [(False, 'zero_shot_tl'), (False, 'fine_tuned_tl'), (True, 'fine_tuned_tl')]

HARDWARE_ACCELERATION = True

## Experiments
This is running 8 experiments each num_experiments times. This comes out to 400 runs for us. 

In [4]:
all_results = {}

for (data_augmentation, embedding) in test_list:
    for dataset in utils.AVAILABLE_SAR_DATASETS[1:]:
        key = str(data_augmentation) + "_" + dataset + "_" + embedding
        print(key)
        experiment_result = experiment_3(
            dataset,
            embedding,
            data_augmentation,
            hardware_acceleration=HARDWARE_ACCELERATION,
        )
        all_results[key] = experiment_result.copy()
        print(experiment_result)
        print()

False_open_sar_ship_zero_shot_tl


Using cache found in /Users/jameschapman/.cache/torch/hub/pytorch_vision_v0.10.0
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Using cache found in /Users/jameschapman/.cache/torch/hub/pytorch_vision_v0.10.0


KeyboardInterrupt: 

In [ ]:
all_results

{'False_open_sar_ship_zero_shot_tl': {'num_points': 772,
  'experiments': 1,
  'mean': 80.97112860892389,
  'std_dev': 0.0},
 'False_fusar_zero_shot_tl': {'num_points': 3127,
  'experiments': 1,
  'mean': 87.56506651243494,
  'std_dev': 0.0},
 'False_open_sar_ship_fine_tuned_tl': {'num_points': 776,
  'experiments': 1,
  'mean': 79.73684210526316,
  'std_dev': 0.0},
 'False_fusar_fine_tuned_tl': {'num_points': 3151,
  'experiments': 1,
  'mean': 86.56891495601172,
  'std_dev': 0.0},
 'True_open_sar_ship_fine_tuned_tl': {'num_points': 757,
  'experiments': 1,
  'mean': 79.53216374269006,
  'std_dev': 0.0},
 'True_fusar_fine_tuned_tl': {'num_points': 3141,
  'experiments': 1,
  'mean': 86.29737609329446,
  'std_dev': 0.0}}

In [ ]:
new_save_path = EXPERIMENT_3_SAVE_PATH + 'all_results_' + str(EXPERIMENT_3_NUM_EXPERIMENTS) 

df = pd.DataFrame.from_dict(all_results, orient='index')
df.to_pickle(new_save_path + '.pkl')